<h1 align=center><font size = 5>Capstone Project - The Battle of Neighborhoods (Week 1 & 2 combined)</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Introduction</a>

2.  <a href="#item2">Data</a>

3.  <a href="#item3">Methodology</a>

4.  <a href="#item4">Results and Discussion</a>

5.  <a href="#item5">Conclusion</a>  
    </font>
    </div>


## 1. Introduction and Background
#### The aim of this project is to find a safe and secure location for someone who is inetersted in opening a specialized fish & chips joint in the city of Seattle, WA, USA. Before establishing a business in any neighborhood, it is important for stakeholders to safety of the location. The aim of this project is to help stakeholders first find safer neighborhoods in Seattle and then find a location in the neighborhood that has no fish & chips joints so their busniness would be profitable.


#### Before we get the data and start exploring it, let's download all the dependencies that we will need:

In [2]:

import numpy as np
import pandas as pd

#Command to install OpenCage Geocoder for fetching Lat and Lng of Neighborhood
!pip install opencage

#Importing OpenCage Geocoder
from opencage.geocoder import OpenCageGeocode

# use the inline backend to generate the plots within the browser
%matplotlib inline 

#Importing Matplot lib and associated packages to perform Data Visualisation and Exploratory Data Analysis
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#Importing folium to visualise Maps and plot based on Lat and Lng
!pip install folium
import folium

#Requests to request web pages by making get requests to FourSquare REST Client
import requests

#To normalise data returned by FourSquare API
from pandas.io.json import json_normalize

#Importing KMeans from SciKit library to Classify neighborhoods into clusters
from sklearn.cluster import KMeans

print('Libraries imported')

Matplotlib version:  3.2.2
     |████████████████████████████████| 93 kB 3.9 MB/s  eta 0:00:01
Libraries imported


## 2. Data
#### Seattle crime data from 2008-present is available on the government website https://data.seattle.gov/Public-Safety/SPD-Crime-Data-2008-Present/tazs-3rd5. The json file which is available on the website is used to create corresposnding dataframe.
#### The dataset consists of neighborhood, type of crime, recorded date/time, location cooordniates and lots of other details which are not necessarily needed for the analysis here. Instead of listing out the properties of the data here, let's load the the dataframe and take a look.

In [88]:
seattle_data = pd.read_json("https://data.seattle.gov/resource/tazs-3rd5.json")
seattle_data.head()

,report_number,offense_id,offense_start_datetime,report_datetime,group_a_b,crime_against_category,offense_parent_group,offense,offense_code,precinct,sector,beat,mcpp,_100_block_address,longitude,latitude,offense_end_datetime
0,2020-044620,12605873663,2020-02-05 10:10:00,2020-02-05 11:24:31,A,SOCIETY,DRUG/NARCOTIC OFFENSES,Drug/Narcotic Violations,35A,W,Q,Q1,MAGNOLIA,32XX BLOCK OF 23RD AVE W,-122.385974,47.649387,NaN
1,2020-044452,12605598696,2020-02-03 08:00:00,2020-02-05 10:06:28,A,PROPERTY,LARCENY-THEFT,Theft of Motor Vehicle Parts or Accessories,23G,N,J,J3,ROOSEVELT/RAVENNA,63XX BLOCK OF 5TH AVE NE,-122.323399,47.675118,2020-02-04 08:00:00
2,2020-044465,12605567653,2020-02-02 20:30:00,2020-02-05 09:39:33,A,PROPERTY,ROBBERY,Robbery,120,N,U,U3,ROOSEVELT/RAVENNA,26TH AVE NE / NE BLAKELEY ST,-122.299552,47.666384,2020-02-02 21:30:00
3,2020-044225,12605174036,2020-02-05 01:17:00,2020-02-05 03:30:55,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,W,Q,Q1,MAGNOLIA,22XX BLOCK OF W RAYE ST,-122.384865,47.642927,2020-02-05 02:21:00
4,2020-044121,12605097782,2020-02-04 22:41:00,2020-02-05 00:59:41,A,PROPERTY,DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY,Destruction/Damage/Vandalism of Property,290,S,R,R1,NORTH BEACON HILL,25XX BLOCK OF 14TH AVE S,-122.314719,47.580248,NaN


#### As can be seen from the data above, there are a lot of attributes that are unnecessary for analysis, so they need to be dropped from the dataframe:

In [89]:
seattle_data.drop(['report_number','offense_id','offense_start_datetime', 'offense_end_datetime', 'group_a_b','crime_against_category','offense_parent_group','offense_code','precinct','sector','beat','_100_block_address', 'longitude', 'latitude'], axis = 1, inplace = True)

#### Let's take a loot at the dataframe now:

In [90]:
seattle_data.head()

,report_datetime,offense,mcpp
0,2020-02-05 11:24:31,Drug/Narcotic Violations,MAGNOLIA
1,2020-02-05 10:06:28,Theft of Motor Vehicle Parts or Accessories,ROOSEVELT/RAVENNA
2,2020-02-05 09:39:33,Robbery,ROOSEVELT/RAVENNA
3,2020-02-05 03:30:55,Destruction/Damage/Vandalism of Property,MAGNOLIA
4,2020-02-05 00:59:41,Destruction/Damage/Vandalism of Property,NORTH BEACON HILL


#### Changing names of properties to something more easy to understand:

In [91]:
seattle_data.rename(columns={'report_datetime':'Date/Time', 'offense': 'Offense', 'mcpp': 'Neighborhood'}, inplace=True)

#### Create a dataframe showing total crimes for different neighborhoods:

In [92]:
seattle_data1 = pd.DataFrame(seattle_data['Neighborhood'].value_counts())
seattle_data1.head()

,Neighborhood
DOWNTOWN COMMERCIAL,85
QUEEN ANNE,68
CAPITOL HILL,62
UNIVERSITY,52
SLU/CASCADE,51


#### Reset index:

In [93]:
seattle_data1.reset_index(inplace=True)
seattle_data1.head()

,index,Neighborhood
0,DOWNTOWN COMMERCIAL,85
1,QUEEN ANNE,68
2,CAPITOL HILL,62
3,UNIVERSITY,52
4,SLU/CASCADE,51


#### Give proper names to columns:

In [95]:
seattle_data1.rename(columns={'Neighborhood':'Crimes', 'index':'Neighborhood'}, inplace=True )
seattle_data1.head()

,Neighborhood,Crimes
0,DOWNTOWN COMMERCIAL,85
1,QUEEN ANNE,68
2,CAPITOL HILL,62
3,UNIVERSITY,52
4,SLU/CASCADE,51


## 2. Methodology
#### Now we have the total number of crimes data for different neighborhoods in Seattle from year 2008 to present. It is fair to say that any neighborhood with 10 or less total crimes for such a long period of time is fairly safe. So the focus now will only be on these neighborhoods.

#### As mentioned above, I am assuming a neighborhood with 10 crimes or less can arbitrarily be considered fairly safe. So let's create correpsnding dataframe:

In [104]:
seattle_data2 = seattle_data1.loc[seattle_data1.Crimes <= 10]
seattle_data2

,Neighborhood,Crimes
31,MADRONA/LESCHI,10
32,SOUTH DELRIDGE,9
33,MORGAN,9
34,PHINNEY RIDGE,9
35,HIGHLAND PARK,9
36,HIGH POINT,8
37,SOUTH BEACON HILL,7
38,LAKEWOOD/SEWARD PARK,7
39,HILLMAN CITY,7
40,EASTLAKE - WEST,7


#### Creating a new consolidated dataset of the Neighborhoods, crime data and the respective Neighbourhood co-ordinates:
#### This data will be fetched using OpenCage Geocoder and explore the neighborhood by plotting it on maps using Folium and perform exploratory data analysis.

#### Let's look at the neighborhoods again:

In [105]:
seattle_data2['Neighborhood'].unique()

array(['MADRONA/LESCHI', 'SOUTH DELRIDGE', 'MORGAN', 'PHINNEY RIDGE',
       'HIGHLAND PARK', 'HIGH POINT', 'SOUTH BEACON HILL',
       'LAKEWOOD/SEWARD PARK', 'HILLMAN CITY', 'EASTLAKE - WEST',
       'CLAREMONT/RAINIER VISTA', 'MONTLAKE/PORTAGE BAY', 'RAINIER VIEW',
       'MILLER PARK', 'SOUTH PARK', 'NEW HOLLY', 'MADISON PARK',
       'PIGEON POINT', 'COLUMBIA CITY', 'RAINIER BEACH', 'ALKI',
       'GENESEE', 'FAUNTLEROY SW', 'UNKNOWN'], dtype=object)

#### Creating a new Data frame with Lat, Lng being fetched from OpenCage geocoder:

In [106]:
Latitude = []
Longitude = []
Neighborhood = seattle_data2['Neighborhood'].unique()
key = '830323b5ca694362904814ff0a11b803'
geocoder = OpenCageGeocode(key)

for i in range(len(Neighborhood)):
    address = '{}, Seattle, King County, Washington, 98164, United States'.format(Neighborhood[i])
    location = geocoder.geocode(address)
    Latitude.append(location[0]['geometry']['lat'])
    Longitude.append(location[0]['geometry']['lng'])
print(Latitude, Longitude)

[47.59829, 47.5521233, 47.6258687, 47.6721306, 47.6038321, 47.5384514, 47.5792579, 47.59829, 47.59829, 47.6408253, 47.59829, 47.59829, 47.598329, 47.59829, 47.5264675, 47.59829, 47.6349784, 47.5711631, 47.5579124, 47.5135339, 47.5762092, 47.5648206, 47.5223995, 47.6038321] [-122.335561, -122.3638741, -122.3216487, -122.3540306, -122.3300624, -122.3773869, -122.3115977, -122.335561, -122.335561, -122.3257232, -122.335561, -122.335561, -122.313409, -122.335561, -122.3229131, -122.335561, -122.2778633, -122.3592519, -122.2852162, -122.2645415, -122.4098507, -122.3679048, -122.3934168, -122.3300624]


#### Let's take a look at the new dataframe with Neighborhoods in Seattle along with centroid of their coordinates:

In [107]:
seattle_dict = {'Neighborhood': Neighborhood,'Latitude': Latitude,'Longitude':Longitude}
seattle_geo = pd.DataFrame(data=seattle_dict, columns=['Neighborhood', 'Latitude', 'Longitude'], index=None)

seattle_geo

,Neighborhood,Latitude,Longitude
0,MADRONA/LESCHI,47.598290,-122.335561
1,SOUTH DELRIDGE,47.552123,-122.363874
2,MORGAN,47.625869,-122.321649
3,PHINNEY RIDGE,47.672131,-122.354031
4,HIGHLAND PARK,47.603832,-122.330062
5,HIGH POINT,47.538451,-122.377387
6,SOUTH BEACON HILL,47.579258,-122.311598
7,LAKEWOOD/SEWARD PARK,47.598290,-122.335561
8,HILLMAN CITY,47.598290,-122.335561
9,EASTLAKE - WEST,47.640825,-122.325723


#### Import other libraries:

In [108]:
!pip install geopandas
!pip install geopy
!pip install tqdm

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook

     |████████████████████████████████| 962 kB 11.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 977 kB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 27.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 46.7 MB/s eta 0:00:01


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`


#### Obataining the geographical coordiantes of Seattle to plot on map:

In [110]:
address = 'Seattle, King County, Washington, 98164, United States'

location = geocoder.geocode(address)
latitude = location[0]['geometry']['lat']
longitude = location[0]['geometry']['lng']

print('The geograpical coordinate of Seattle, WA are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Seattle, WA are 47.6038321, -122.3300624.


#### Using Folium to plot Seattle city with its Neighborhoods:

In [111]:
seattle_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(seattle_geo['Latitude'], seattle_geo['Longitude'], seattle_geo['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(seattle_map)  
    
seattle_map

#### Creating a new consolidated dataset of the Neighborhoods, and the most common venues and the respective neighborhood along with coordinates:

#### Setting up Fousquare credentials:

In [112]:
CLIENT_ID = 'S4DEDDFLX1JHKSRC20WYKDJGCLU3UCPPOG3LQKDSJGJFJ5HZ' # Foursquare ID
CLIENT_SECRET = '2NTRF0IHR0GESDR0LGMMSATTBOJCXFRWAWQUOC3AYPWCGRHL' # Foursquare Secret
VERSION = '20180605'
LIMIT = 200
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: S4DEDDFLX1JHKSRC20WYKDJGCLU3UCPPOG3LQKDSJGJFJ5HZ
CLIENT_SECRET:2NTRF0IHR0GESDR0LGMMSATTBOJCXFRWAWQUOC3AYPWCGRHL


#### Defining a function to fetch top 10 venues around a given neighborhood:

In [113]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Category']
    
    return(nearby_venues)

#### Generate Seattle venues:

In [114]:
seattle_venues = getNearbyVenues(names=seattle_geo['Neighborhood'],
                                   latitudes=seattle_geo['Latitude'],
                                   longitudes=seattle_geo['Longitude']
                                  )

MADRONA/LESCHI
SOUTH DELRIDGE
MORGAN
PHINNEY RIDGE
HIGHLAND PARK
HIGH POINT
SOUTH BEACON HILL
LAKEWOOD/SEWARD PARK
HILLMAN CITY
EASTLAKE - WEST
CLAREMONT/RAINIER VISTA
MONTLAKE/PORTAGE BAY
RAINIER VIEW
MILLER PARK
SOUTH PARK
NEW HOLLY
MADISON PARK
PIGEON POINT
COLUMBIA CITY
RAINIER BEACH
ALKI
GENESEE
FAUNTLEROY SW
UNKNOWN


#### Data frame containing venues for each neighborhood:

In [115]:
print(seattle_venues.shape)
seattle_venues.head()

(1042, 5)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,MADRONA/LESCHI,47.59829,-122.335561,General Porpoise Coffee & Doughnuts,Donut Shop
1,MADRONA/LESCHI,47.59829,-122.335561,Il Terrazzo Carmine,Italian Restaurant
2,MADRONA/LESCHI,47.59829,-122.335561,Salumi,Italian Restaurant
3,MADRONA/LESCHI,47.59829,-122.335561,Taylor Shellfish Oyster Bar,Seafood Restaurant
4,MADRONA/LESCHI,47.59829,-122.335561,Caffè Umbria,Coffee Shop


#### Number of venues per neighborhood:

In [116]:
seattle_venues.groupby('Neighborhood').count().drop(['Neighborhood Latitude','Neighborhood Longitude','Venue Category'], axis = 1)

,Venue
Neighborhood,
ALKI,41
CLAREMONT/RAINIER VISTA,78
COLUMBIA CITY,36
EASTLAKE - WEST,29
FAUNTLEROY SW,10
GENESEE,5
HIGH POINT,5
HIGHLAND PARK,96
HILLMAN CITY,78


In [117]:
print('There are {} uniques categories.'.format(len(seattle_venues['Venue Category'].unique())))

There are 149 uniques categories.


## 3. Modelling
#### Will be running k-means machine learning algorithm to cluster the neighborhood into 5 clusters

#### One Hot Encoding to analyze each neighborhood:

In [118]:
# one hot encoding
seattle_onehot = pd.get_dummies(seattle_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
seattle_onehot['Neighborhood'] = seattle_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

seattle_onehot.head()

,Zoo Exhibit,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,...,Toy / Game Store,Track,Trail,Tram Station,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
seattle_onehot.shape

(1042, 149)

In [120]:
seattle_grouped = seattle_onehot.groupby('Neighborhood').mean().reset_index()
seattle_grouped

,Neighborhood,Zoo Exhibit,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Toy / Game Store,Track,Trail,Tram Station,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio,Zoo
0,ALKI,0.000000,0.000000,0.000000,0.000000,0.000000,0.048780,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.024390,0.00,0.000000,0.024390,0.000000,0.000000,0.000000,0.000000
1,CLAREMONT/RAINIER VISTA,0.000000,0.000000,0.000000,0.012821,0.000000,0.025641,0.012821,0.000000,0.000000,...,0.012821,0.000000,0.000000,0.00,0.000000,0.000000,0.012821,0.000000,0.012821,0.000000
2,COLUMBIA CITY,0.000000,0.055556,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.027778,0.000000,0.027778,0.000000,0.000000
3,EASTLAKE - WEST,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000
4,FAUNTLEROY SW,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,GENESEE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,HIGH POINT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,HIGHLAND PARK,0.000000,0.000000,0.000000,0.010417,0.000000,0.010417,0.010417,0.000000,0.010417,...,0.010417,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010417,0.000000
8,HILLMAN CITY,0.000000,0.000000,0.000000,0.012821,0.000000,0.025641,0.012821,0.000000,0.000000,...,0.012821,0.000000,0.000000,0.00,0.000000,0.000000,0.012821,0.000000,0.012821,0.000000
9,LAKEWOOD/SEWARD PARK,0.000000,0.000000,0.000000,0.012821,0.000000,0.025641,0.012821,0.000000,0.000000,...,0.012821,0.000000,0.000000,0.00,0.000000,0.000000,0.012821,0.000000,0.012821,0.000000


#### Top 5 most common venues across neighborhoods:

In [121]:
num_top_venues = 5

for hood in seattle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = seattle_grouped[seattle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALKI----
                venue  freq
0      Ice Cream Shop  0.12
1  Mexican Restaurant  0.07
2         Art Gallery  0.05
3     Thai Restaurant  0.05
4         Coffee Shop  0.05


----CLAREMONT/RAINIER VISTA----
                    venue  freq
0             Coffee Shop  0.09
1            Cocktail Bar  0.05
2              Sports Bar  0.04
3                    Park  0.04
4  Thrift / Vintage Store  0.03


----COLUMBIA CITY----
                venue  freq
0         Coffee Shop  0.08
1                 Bar  0.06
2         Pizza Place  0.06
3  African Restaurant  0.06
4      Ice Cream Shop  0.06


----EASTLAKE - WEST----
                venue  freq
0         Coffee Shop  0.17
1      Sandwich Place  0.14
2  Italian Restaurant  0.10
3          Playground  0.07
4         Pizza Place  0.07


----FAUNTLEROY SW----
                 venue  freq
0        Boat or Ferry   0.2
1                 Park   0.2
2  American Restaurant   0.1
3          Bus Station   0.1
4          Pizza Place   0.1


----GEN

#### Now let's create a new dataframe and display the top 10 venues for each neighborhood:

In [122]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [125]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALKI,Ice Cream Shop,Mexican Restaurant,Italian Restaurant,Park,Thai Restaurant,Art Gallery,Coffee Shop,Market,Sports Bar,Pizza Place
1,CLAREMONT/RAINIER VISTA,Coffee Shop,Cocktail Bar,Park,Sports Bar,Thrift / Vintage Store,Pub,Lounge,Italian Restaurant,Sushi Restaurant,Pizza Place
2,COLUMBIA CITY,Coffee Shop,Pizza Place,African Restaurant,Bar,Ice Cream Shop,Diner,Indie Movie Theater,Sports Bar,Mexican Restaurant,Italian Restaurant
3,EASTLAKE - WEST,Coffee Shop,Sandwich Place,Italian Restaurant,Pizza Place,Playground,Harbor / Marina,Ethiopian Restaurant,Spa,Mexican Restaurant,Scenic Lookout
4,FAUNTLEROY SW,Park,Boat or Ferry,Market,Bus Station,Bakery,Pier,American Restaurant,Pizza Place,Donut Shop,Dry Cleaner


#### Cluster neighborhoods:

In [126]:
# set number of clusters
kclusters = 5

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 2, 1, 0, 0, 0], dtype=int32)

In [127]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

seattle_merged = seattle_geo

# merge toronto_grouped with Vancouver data to add latitude/longitude for each neighborhood
seattle_merged = seattle_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

seattle_merged.head()

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,MADRONA/LESCHI,47.598290,-122.335561,0,Coffee Shop,Cocktail Bar,Park,Sports Bar,Thrift / Vintage Store,Pub,Lounge,Italian Restaurant,Sushi Restaurant,Pizza Place
1,SOUTH DELRIDGE,47.552123,-122.363874,0,Gas Station,Park,Pizza Place,Vietnamese Restaurant,Restaurant,Trail,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,MORGAN,47.625869,-122.321649,0,Cocktail Bar,Italian Restaurant,Coffee Shop,American Restaurant,Bus Stop,Bar,Men's Store,Spa,Café,Restaurant
3,PHINNEY RIDGE,47.672131,-122.354031,0,Zoo Exhibit,Bar,Pizza Place,Dessert Shop,Japanese Restaurant,Pub,Playground,Pie Shop,Park,New American Restaurant
4,HIGHLAND PARK,47.603832,-122.330062,0,Coffee Shop,Hotel,Café,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Salad Place,Donut Shop,Mexican Restaurant,Hotel Bar


In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(seattle_merged['Latitude'], seattle_merged['Longitude'], seattle_merged['Neighborhood'], seattle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Analysis: 
#### Examining clusters:

In [130]:
#Cluster 1
seattle_merged.loc[seattle_merged['Cluster Labels'] == 0, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,47.598290,Cocktail Bar,Park,Sports Bar,Thrift / Vintage Store,Pub,Lounge,Italian Restaurant,Sushi Restaurant,Pizza Place
1,47.552123,Park,Pizza Place,Vietnamese Restaurant,Restaurant,Trail,Eastern European Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant
2,47.625869,Italian Restaurant,Coffee Shop,American Restaurant,Bus Stop,Bar,Men's Store,Spa,Café,Restaurant
3,47.672131,Bar,Pizza Place,Dessert Shop,Japanese Restaurant,Pub,Playground,Pie Shop,Park,New American Restaurant
4,47.603832,Hotel,Café,Cocktail Bar,Japanese Restaurant,Italian Restaurant,Salad Place,Donut Shop,Mexican Restaurant,Hotel Bar
6,47.579258,Café,Coffee Shop,Gas Station,Brewery,Mediterranean Restaurant,Scenic Lookout,Taco Place,Food Truck,Fried Chicken Joint
7,47.598290,Cocktail Bar,Park,Sports Bar,Thrift / Vintage Store,Pub,Lounge,Italian Restaurant,Sushi Restaurant,Pizza Place
8,47.598290,Cocktail Bar,Park,Sports Bar,Thrift / Vintage Store,Pub,Lounge,Italian Restaurant,Sushi Restaurant,Pizza Place
9,47.640825,Sandwich Place,Italian Restaurant,Pizza Place,Playground,Harbor / Marina,Ethiopian Restaurant,Spa,Mexican Restaurant,Scenic Lookout
10,47.598290,Cocktail Bar,Park,Sports Bar,Thrift / Vintage Store,Pub,Lounge,Italian Restaurant,Sushi Restaurant,Pizza Place


In [131]:
#Cluster 2
seattle_merged.loc[seattle_merged['Cluster Labels'] == 1, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,47.538451,Playground,Field,Spa,Eye Doctor,Zoo,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [132]:
#Cluster 3
seattle_merged.loc[seattle_merged['Cluster Labels'] == 2, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,47.564821,Gym,Soccer Field,Golf Course,Zoo,Event Space,Field,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [133]:
#Cluster 4
seattle_merged.loc[seattle_merged['Cluster Labels'] == 3, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,47.513534,Garden,Baseball Field,Fast Food Restaurant,Falafel Restaurant,Flower Shop,Fish & Chips Shop,Field,Farmers Market,Eye Doctor


In [134]:
#Cluster 5
seattle_merged.loc[seattle_merged['Cluster Labels'] == 4, seattle_merged.columns[[1] + list(range(5, seattle_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,47.526468,Library,Park,Bar,Zoo,Farmers Market,Flower Shop,Fish & Chips Shop,Field,Fast Food Restaurant


## 4. Results and Discussion
#### The objective of the problem was to help stakeholders identify safer neighborhoods in Seattle, WA, USA a commercial establishment especially fish & chips joint. This has been achieved by first making use of Seattle crime data to identify fairly safer neighborhoods for any business to be viable. It was imperative to choose the neighborhood where an eatery/restaurant that is specialized on fish & chips were not among venues in a close proximity to each other. We achieved this by grouping the neighborhoods into clusters to assist the stakeholders by providing them with relavent data about venues and safety of a given neighborhood.

## 5. Conclusion

#### We have explored the crime data to understand different types of crimes in all neighborhoods of the city of Seattle, WA, USA. Using machine learning mdel, we have further shortlisted the neighborhoods based on the common venues, to choose a neighborhood which best suits the business problem.
